In [1]:
import pandas as pd
import numpy as np
import random

## Correlation of metrics
Emotion - Moderate-Highcorrelation with engagement, behaviors and attentiveness

HeadGaze - Moderate-High Correlation, facing front fairly justify attentivness, looking away is obvious disengagement

Motion - Low-Moderate Correlation, increasing micro-motions shows correlation with attentivness (Reseach Article)

HandPose - High Correlation, raising hands definitely engagement with current session 

SleepPose - High correlation, detected = not engaged

Correlation ranking
1. HandPose
2. SleepPose
3. Emotion
4. HeadGaze
5. Motion

### Futhermore, we split the various metrices into their individual components/ranking
1. HandPose (1 if raised, 0 if not)
2. SleepPose (1 if sleep, 0 if not)
3. Emotion

    a. Happy

    b. Angry
    
    c. Disgusted
    
    d. Fearful
    
    e. Neutral
    
    f. Sad
    
    g. Surprised
    
    
4. HeadGaze (2 if facing front, 1 if sideways, 0 if back)

5. Motion tbc

In [2]:
categories = ['Happy', 'Angry', 'Disgusted', 'HandRaised', 'Sleep', 'HeadGaze', 'Engagement']
emo_list = ['Happy', 'Angry', 'Disgusted']

df = pd.DataFrame(index = range(0,3000), columns = categories)

In [3]:
def inputEmotions(df, emo_list):
    for row in range(3000):
        #generate a random probability function (array) which adds up to one, returns a list of list after .tolist()
        randArray = np.array(np.random.dirichlet(np.ones(3), size = 1)).tolist()[0]
        for i in range(3):
            df[emo_list[i]].iloc[row] = randArray[i] 

In [4]:
#we split the categories here carefully because they shouldnt conflict

def _handRaised(x):
    return 1

def _handMid(x):
    return random.randint(0,1)

def _handNotRaised(x):
    return 0

def _handRandom(x):
    return random.randint(0,1)

def _sleep(x):
    return 1

def _notSleep(x):
    return 0

def _sleepRandom(x):
    return random.randint(0,1)

def _headFront(x):
    return 2

def _headModerate(x):
    return random.randint(1,2)

def _headNotFront(x):
    return random.randint(0,1)

def _headRandom(x):
    return random.randint(0,2)

def _engagedHigh(x):
    return random.randint(4,5)

def _engagedMid(x):
    return random.randint(2,3)

def _engagedLow(x):
    return random.randint(1,1)

def _engagedNoise(x):
    return random.randint(1,5)

## Synthetic Data
1. We work with 3000 training examples with this neural network
2. Assuming 10% noise in data, we work with 2700 reliable datasets, 300 noise datsets (random labels)
3. Breaking down the 2700 training examples

#### we break into high, middle and low tier ratings with a few simple rules in place to guide the model to learn
1. high - high happy probability, moderate-low angry/disgusted, handraised 1, sleep 0, headgaze 2
2. mid - moderate emotions probability, handraise random, sleep 0, headgaze 1-2
3. low - low happy probability, moderate-high angry/disgusted, handraised 0, sleep 0/1, headgaze 0,1
4. noise - all are noise
5. engagement - we will take softmax of range 1 to 5, 4-5 being most engaged, 2-3 being mildly engaged, 1 least engaged

In [5]:
inputEmotions(df , emo_list)

In [6]:
#sort df by happy vs not happy
df = df.sort_values(by = ['Happy'], ascending = False)

In [7]:
df.head()

,Happy,Angry,Disgusted,HandRaised,Sleep,HeadGaze,Engagement
1815,0.982306,0.00102089,0.016673,NaN,NaN,NaN,NaN
2596,0.97358,0.0102074,0.0162124,NaN,NaN,NaN,NaN
1388,0.970961,0.00670249,0.0223362,NaN,NaN,NaN,NaN
968,0.964519,0.00524098,0.0302397,NaN,NaN,NaN,NaN
467,0.962101,0.0357958,0.00210353,NaN,NaN,NaN,NaN


##### We will sort 900 rows of each tier (high, mid, low), along with 100 each of noise into the NN

In [8]:
#High
df['HandRaised'][:900] = df['HandRaised'][:900].apply(lambda x: _handRaised(x))
df['HandRaised'][900:1000] = df['HandRaised'][900:1000].apply(lambda x: _handRandom(x))
df['Sleep'][:900] = df['Sleep'][:900].apply(lambda x: _notSleep(x))
df['Sleep'][900:1000] = df['Sleep'][900:1000].apply(lambda x: _sleepRandom(x))
df['HeadGaze'][:900] = df['HeadGaze'][:900].apply(lambda x: _headFront(x))
df['HeadGaze'][900:1000] = df['HeadGaze'][900:1000].apply(lambda x: _headRandom(x))

df['Engagement'][:900] = df['Engagement'][:900].apply(lambda x: _engagedHigh(x))
df['Engagement'][900:1000] = df['Engagement'][900:1000].apply(lambda x: _engagedNoise(x))

#Mid
df['HandRaised'][1000:1900] = df['HandRaised'][1000:1900].apply(lambda x: _handMid(x))
df['HandRaised'][1900:2000] = df['HandRaised'][1900:2000].apply(lambda x: _handRandom(x))
df['Sleep'][1000:1900] = df['Sleep'][1000:1900].apply(lambda x: _notSleep(x))
df['Sleep'][1900:2000] = df['Sleep'][1900:2000].apply(lambda x: _sleepRandom(x))
df['HeadGaze'][1000:1900] = df['HeadGaze'][1000:1900].apply(lambda x: _headModerate(x))
df['HeadGaze'][1900:2000] = df['HeadGaze'][1900:2000].apply(lambda x: _headRandom(x))

df['Engagement'][1000:1900] = df['Engagement'][1000:1900].apply(lambda x: _engagedMid(x))
df['Engagement'][1900:2000] = df['Engagement'][1900:2000].apply(lambda x: _engagedNoise(x))

#Low
df['HandRaised'][2000:2900] = df['HandRaised'][2000:2900].apply(lambda x: _handNotRaised(x))
df['HandRaised'][2900:3000] = df['HandRaised'][2900:3000].apply(lambda x: _handNotRaised(x))
df['Sleep'][2000:2900] = df['Sleep'][2000:2900].apply(lambda x: _sleepRandom(x))
df['Sleep'][2900:3000] = df['Sleep'][2900:3000].apply(lambda x: _sleepRandom(x))
df['HeadGaze'][2000:2900] = df['HeadGaze'][2000:2900].apply(lambda x: _headNotFront(x))
df['HeadGaze'][2900:3000] = df['HeadGaze'][2900:3000].apply(lambda x: _headRandom(x))

df['Engagement'][2000:2900] = df['Engagement'][2000:2900].apply(lambda x: _engagedLow(x))
df['Engagement'][2900:3000] = df['Engagement'][2900:3000].apply(lambda x: _engagedNoise(x))

C:\Users\Gabriel Sze\Anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gabriel Sze\Anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gabriel Sze\Anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gabriel Sze\Anaconda3\envs\tensor\lib\si

In [9]:
df

,Happy,Angry,Disgusted,HandRaised,Sleep,HeadGaze,Engagement
1815,0.982306,0.00102089,0.016673,1,0,2,5
2596,0.97358,0.0102074,0.0162124,1,0,2,5
1388,0.970961,0.00670249,0.0223362,1,0,2,5
968,0.964519,0.00524098,0.0302397,1,0,2,5
467,0.962101,0.0357958,0.00210353,1,0,2,5
...,...,...,...,...,...,...,...
2732,0.000581084,0.674897,0.324522,0,1,2,3
1521,0.000321334,0.403506,0.596173,0,1,1,2
2272,0.000265595,0.181587,0.818147,0,1,0,1
2744,0.000227761,0.858712,0.14106,0,1,0,4


In [10]:
df.to_excel('EngagementTest4.xlsx')